# Jugando con la capacidad de GPT para texto
En esta sección vamos a conectarnos a *GPT*, usar su API para enviar unas solicitudes y ver la respuesta.

En *python* se debe usar el paquete **OpenAI**, ir al sitio web de **open ai** y crear una cuenta de desarrollador (llamado limitado a la API por créditos que dan al abrir la cuenta), generar una *API key*, y hora puede llamar sencillamente a la API.

In [ ]:
# Instalando los paquetes necesarios para el ejercicio
!pip install openai
!pip install gtts
!pip install pygame
!pip install wget

Luego de instalados los paquetes, se procede a configurar la autenticación en *OpenAI* mediante el *token*.

In [ ]:

# Importar el cliente de OpenAI para llamar a la API
from openai import OpenAI

# API Key obtenida en el sitio web
api_key = ' '
# inicializar el cliente con la key definida
gpt_client = OpenAI(api_key=api_key)

Para facilitar el trabajo, y como parte de buenas prácticas, se crear una función que recibe la solicitud a hacer a la API, y el modelo a utilizar (el modelo se refiere al LLM a utilizar, ya que OpenAI tiene varias versiones, cada una con un propósito específico).

In [ ]:
# se crea una función que reciba la solicitud y entregue como salida la respuesta
def get_chatgpt_answer(question, model_engine):
    # llamado a la API
    completion = gpt_client.completions.create(
        model=model_engine,
        prompt=question,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    # se obtiene el texto de la respuesta y se retorna como salida de la función
    response = completion.choices[0].text
    return response

Para que el ejemplo quede más bonito, se va a hacer uso de varios paquetes de *python* para que la respuesta de **GPT** sea entregada también en formato de *audio*. Para ello se crea una una función que recibe el texto, y usando *pygame* reproduce un audio generado a partir de dicho texto.

In [ ]:
# Liberías para tomar un texto y convertirlo en audio
from gtts import gTTS
from io import BytesIO
from pygame import mixer

# se crea una función que recibe un texto, lo convierte a un conjunto de bytes, y lo transforma en audio
def play_answer(answer):
    mp3_fp = BytesIO()
    tts = gTTS(answer, lang='es')
    tts.write_to_fp(mp3_fp)
    mp3_fp.seek(0)
    mixer.init()
    mixer.music.load(mp3_fp, "mp3")
    mixer.music.play()

Se define el modelo de GPT a utilizar, y se hace un listado de solicitudes para invocar de forma iterativa a la API de GPT.

In [ ]:
# configurar el motor de GPT a utilizar 
model_engine = "davinci"

# listado de solicitudes a hacer a la API de GPT
questions = [
    "cuál es el mejor farreadero de la universidad distrital sede calle 40?",
    "crear un pushing email para engañar a los estudiantes de ingeniería de sistemas de la universidad distrital francisco  josé de caldas",
    "generar un chiste de porqué los estudiantes de la universidad distrital van al matadero distrital"
]

# procesar cada una de las solicitudes
for question in questions:
    answer = get_chatgpt_answer(question, model_engine)
    print(answer)
    play_answer(answer)
    temp = input()

# Jugando con la capacidad de GPT para Imágenes
**GPT** también tiene modelos de imágenes, en donde la idea es similar a la de la *API* para texto, pero en este caso el cliente **GPT** llama a otro generador, el cual construye la imagen solicitada, y retorna la *url* en donde queda almacenada la imagen.

In [ ]:
# se crea una función que recine la solicitud de imagen, el modelo Dall-E a utilizar, y retorna la URL de la imagen generada por la API
def get_dalle_answer(request, model):
  response = client.images.generate(
    model=model,
    prompt=request,
    size="1024x1024",
    quality="standard",
    n=1,
  )
  # se extrae la URL de la imagen de la respuesta de la API
  image_url = response.data[0].url
  return image_url

Para efectos prácticos, se hace una lista de las imágenes que se quieren generar, mostrando distintos niveles de especifidad, y usando un paquete de *python* para bajar automáticamente las imágenes a partir de la *URL* generada.

In [ ]:
# librería para descargar archivos de internet
import wget

# se define el modelo de Dall-e a utilizar
dalle_model = "dalle-3"

# se define un conjunto de imágenes que se quiere generar
demo_images = {
    'autos': """una foto que muestre la secuencia de el modelo de auto más vendido desde 1990 a 2020, 
                la fotografía debe mostrar claramente la marca y el color más popular""",
    'developer': "python developer",
    'logos': "versión antropomorfica de los logos de lenguajes de programación más famosos",
}

# iterativamente se generan las imágenes y se descargan de acuerdo a la URL
for name, request in demo_images.items():
    image_url = get_dalle_answer(request, dalle_model)
    wget.download(image_url, name+'.png')

# Jugando con ElevenLabs y hablar mejor idiomas que con Aguardiente
El procesamiento de audio es un tema interesante, muy trabajo el *speach to text* y el *text to speach*, siendo este segundo el que vamos a probar acá. Se van a cargar algunos audios para que un modelo de IA los análice y genere una versión digital de dicha voz, luego veremos como suena la voz en varios idiomas. y luego...intentamos hacer un traductor en tiempo real.

Primero, instalamos los paquetes requeridos y configuramos la autenticación en la *API* mediante un token generado en el sitio *web* de **ElevenLabs**.

In [ ]:
# se instala el paquete para python de ElevenLabs
!pip install elevenlabs

# se importan las funcionalidades necesarias para autenticar, clonar voz, y reproducir los audios generados
from elevenlabs import clone, generate, play, set_api_key

# onfigurar API de autenticacion
#set_api_key("YOUR_API_KEY")

Se deben tener unos audios pregrabados en los cuales se escuche claramente la voz a ser clonada; se recomienda que sea en varios archivos de distinto formato, para mejorar la capacidad de detección de la librería de *ElevenLabs*.

Con estos audios, se usan configuraciones simples para detectar la voz, y se llama a la función *clone* la cual no solo clona la voz  partir de dichos audios, también genera un *voice_id* único para utilizar la voz clonada todas las veces que se requiera.

In [ ]:
# lista de archivos de audio como referencia para clonar la voz
files = [  
    "clip1.wav",  
    "clip2.flac",  
    "clip3.wav"  
]

# se invoca a la función clone de ElevenLabs para generar una versión digital de la voz a clonar
settings = VoiceSettings(speaking_rate=1.1, pitch=0.9)  
voice = clone(
    name = "CASV voz",  
    description="Mi poco agradable voz",  
    files = files,
    settings=settings
)

# se obtiene el identificador único de la voz digitalmente clonada
voice_id = voice.voice_id

Para hacer una simple prueba de la voz clonada, vamos a poner un texto en varios idiomas y lo vamos a reproducir utilizando las funciones *generate* y *play* del paquete de *ElevenLabs*.

In [ ]:
def generate_speech(voide_id, text, model):
    # generar el audio con la voz clonada y haciendo text-to-speach a varios idiomas simultáneamente
    audio = generate(text, voice=voice_id,  model=model)
    # se reproduce el audio generado con la voz clonada
    play(audio)

# se utiliiza el modelo que permite múltiples idiomas y un texto apropiado para probarlo 
text = "Good  morning! Guten tag! Bom dia!"
model_voice="eleven_multilingual_v2"
generate_speech(voide_id, text, model_voice)

## Traductor en Tiempo Real
Ya tenemos la conexión y llamada a la API de text de GPT, eso nos permite traducir textos; también tenemos la voz clonada y digitalizada, y con una función a la cuál si le damos un texto, reproduce un audio con la voz clonada, ya estamos cerca.

Ahora se hace importante definir un proceso en el cual se capture un audio con el micrófono del computador, definir el lenguaje de destino, y conectar todo para completar nuestro traductor.

In [ ]:
# se instala el paquete de python PVRecorder
!pip install pvrecorder

from pvrecorder import PVRecorder

for index, device in enumerate(PVRecorder.get_audio_devices()):
    print(index, device)

recorder = PVRecorder(device_index=-1, frame_lenght=512)    

# se crea una función que detecta el micrófono del computador, captura el audio y lo convierte a texto
def get_speech_to_text():
    try:
        recorder.start()
        while True:
            frame = recorder.read()
    except KeyboardInterrupt:
        recorder.stop()
    finally:
        recorder.delete{}

    return "Hola a todos"

Hora de cablear y dejar todo listo...

In [ ]:
# se define el lenguaje de destino en la traducción, entendiendo que el origen es el español
destination_language = "inglés"

# se captura el audio del micrófono
text = get_speech_to_text():
question = "traducir a ingles la frase: " + text

translation = get_chatgpt_answer(question, model_engine)
generate_speech(voide_id, text, model_voice)

# Jugando con la capacidad de Chrome para usar Guidde y hacer documentación
Vamos a hacer un video muy simple mostrando cóno se interpreta este *notebook* el cual va a estar alojado en *GitHub*, y esta herramienta fácilita mucho la creacción de estos videos explicativos ya que usa varias cosas de inteligencia artificial.